In [1]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
import numpy as np
from sklearn.model_selection import StratifiedKFold
from config_file import config
import pandas as pd
import Train_model
from tensorflow.compat.v1 import ConfigProto 
from tensorflow.compat.v1 import InteractiveSession 
import tensorflow as tf 
 
from keras import backend

importing Jupyter notebook from config_file.ipynb
importing Jupyter notebook from Train_model.ipynb
importing Jupyter notebook from model.ipynb


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 9, 9, 1)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 32)          320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

In [2]:
classes = config["run"]["classes"]
n_splits_k = config["Train_model"]["n_splits_k"]

  


Compile and train the model

In [3]:
def run_main():

    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(gpus)
#     configPro = ConfigProto()
#     configPro.gpu_options.allow_growth = True
#     session = InteractiveSession(config=configPro)
#     gpu_options = GPUOptions(per_process_gpu_memory_fraction=0.33)
#     sess = Session(config=ConfigProto(gpu_options=gpu_options))
   #os.environ['CUDA_VISIBLE_DEVICES'] ='-1'
    datasets = pd.read_csv("data/data_total3.csv") 
    features = datasets.iloc[:,:-1]
    labels =  datasets.iloc[:,-1]
    skf = StratifiedKFold(n_splits=n_splits_k,random_state = 2)
    Test_accuracy_Epo = []
    Accuracy_value_Epo = []
    Accuracy_value_all = []
    Loss_value_all = []
    AccofB_test = []
    AccofA_test = []
    epoch = 0
    for train_index, test_index in skf.split(features, labels):  
        features_train = features.iloc[train_index]
        features_test =  features.iloc[test_index]
        labels_train = labels.iloc[train_index]
        labels_test =  labels.iloc[test_index]
        features_train = features_train.values
        labels_train = labels_train.values
        features_test = features_test.values
        labels_test = labels_test.values
        labels_train = labels_train.reshape(len(labels_train))
        labels_test = labels_test.reshape(len(labels_test))
        print("Number of examples in the train set:",labels_train.shape)
        print("Number of examples in the test set:",labels_test.shape)
        configPro = ConfigProto()
        configPro.gpu_options.allow_growth = True
        session = InteractiveSession(config=configPro)
        history = Train_model.train_model(features_train,labels_train)
        print(history.history.keys())
        Accuracy_value,acc,Loss_value,Epoch_accracy 
        Accuracy_value_Epo.append(Epoch_accracy)
        Accuracy_value_all.extend(Accuracy_value)
        Loss_value_all.extend(Loss_value)
        print("Epoch_accracy:",Epoch_accracy)
        print("Accuracy_value_Epo",Accuracy_value_Epo)
        plt.figure(figsize=(20, 10))
        plt.plot(range(Train_model.Iteration*(epoch+1)), Accuracy_value_all)
        plt.xlabel('training step')
        plt.ylabel('accuracy value')
        plt.title('the accuracy value of training data')
       # plt.savefig('model_folder/results/accuracy.png')
        plt.show()

        plt.figure()
        plt.plot(range(Train_model.Iteration*(epoch+1)), Loss_value_all)
        plt.xlabel('training value')
        plt.ylabel('loss value')
        plt.title('the value of the loss function of the training data')
       # plt.savefig('model_folder/results/loss.png')
        plt.show()

        plt.figure()
        plt.plot(range(epoch+1),Accuracy_value_Epo)
        plt.xlabel('Epoch')
        plt.ylabel('accracy')
        plt.title('Epoch average accuracy')
       # plt.savefig('model_folder/results/Epoch_accuracy.png')
        plt.show()

        plt.figure()
        plt.plot(classes, acc)
        plt.ylabel('accuracy')
        plt.xlabel('classification')
        plt.title('Accuracy of each classification')
        #plt.savefig('model_folder/results/Accuracy_of_each_classification.png')
        plt.show()

        precision,accBA =evaluation(features_test,labels_test)
        Test_accuracy_Epo.append(precision)
        AccofB_test.append(accBA[0])
        AccofA_test.append(accBA[1])
        plt.figure()
        plt.plot(range(epoch+1),Test_accuracy_Epo)
        plt.ylabel('Test_accuracy')
        plt.xlabel('Epoch')
        plt.title('Test_accuracy')
       # plt.savefig('model_folder/results/test_accuracy.png')
        plt.show()

        y = list(range(0,epoch+1))
        plt.bar(x=range(epoch+1), height = AccofB_test, width=width, label='BENIGN',color = 'steelblue',alpha = 0.8)  
        plt.bar(x=np.arange(epoch+1)+width,height = AccofA_test, width=width, label='Attack',color = 'indianred',alpha = 0.8)  
        plt.xticks(np.arange(len(y))+width/2, y)
        plt.ylabel('Test_accuracyofclasses')
        plt.xlabel('Epoch')
        plt.title('Test_accuracyofclasses')
        plt.legend()  
       # plt.savefig('model_folder/results/test_accuracyofclasses.png')
        plt.show() 
        epoch += 1
    average_train_accracy=model.Epoch_accuracy_mean(Epoch_accracy)
    print("average_train_accracy is:",average_train_accracy)
    average_test_accracy=model.Epoch_accuracy_mean(Test_accuracy_Epo)
    print("average_test_accracy is:",average_test_accracy)

In [4]:
if __name__ =="__main__":
    run_main()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Number of examples in the train set: (976972,)
Number of examples in the test set: (139568,)
kkkkkkkk
Train on 781577 samples, validate on 195395 samples
Epoch 1/3


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_3/convolution (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_1561]

Function call stack:
keras_scratch_graph
